In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
darts = pd.read_csv("datasets/darts.csv")

# Transform into a categorical variable
darts.competitor = pd.Categorical(darts.competitor)

# Assign a number to each category (label encoding)
darts.competitor = darts.competitor.cat.codes

coordinates = darts.drop(["competitor"], axis=1)

# Use to_categorical on your labels
competitors = to_categorical(darts.competitor)

In [ ]:
coord_train, coord_test, competitors_train, competitors_test = train_test_split(
    coordinates, competitors, test_size=0.3, random_state=111
)

In [ ]:
early_stopping = EarlyStopping(monitor="val_loss", patience=5)

In [ ]:
# Instantiate a sequential model
model = Sequential()

# Add 3 dense layers of 128, 64 and 32 neurons each
model.add(Dense(128, input_shape=(2,), activation="relu"))
model.add(Dense(64, activation="relu"))
model.add(Dense(32, activation="relu"))

# Add a dense layer with as many neurons as competitors
model.add(Dense(4, activation="softmax"))

# Compile your model using categorical_crossentropy loss
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

training_model = model.fit(
    coord_train,
    competitors_train,
    validation_data=(coord_test, competitors_test),
    epochs=100,
    callbacks=[
        early_stopping,
        ModelCheckpoint("Saved/best_dart_model.hdf5", save_best_only=True),
    ],
)

In [ ]:
# Evaluate your model accuracy on the test data
loss, accuracy = model.evaluate(coord_test, competitors_test)

print("Loss:", loss)
print("Accuracy:", accuracy)

In [ ]:
# Predict
preds = model.predict(coord_test)

# Print preds vs true values
print("{:45} | {}".format("Raw Model Predictions", "True labels"))
for i, pred in enumerate(preds):
    print("{} | {}".format(pred, competitors_test[i]))

In [ ]:
print(training_model.history["val_accuracy"])

In [ ]:
# Plot train vs test accuracy during training
plt.plot(training_model.history["accuracy"], color="b")
plt.plot(training_model.history["val_accuracy"], color="r")
plt.show()